# 🚀 ViT-FishID: Resume Training from Epoch 19

**COLAB PRO EXTENDED TRAINING**
- Resume from: Epoch 19 checkpoint
- Target epochs: 100 total epochs (81 remaining)
- Expected training time: 6-8 hours with Colab Pro
- GPU: Tesla T4/V100/A100 (depending on availability)

This notebook will:
1. ✅ Resume training from your saved checkpoint at epoch 19
2. ✅ Train for 100 total epochs (81 more epochs)
3. ✅ Save checkpoints to Google Drive every 10 epochs
4. ✅ Use semi-supervised learning with your fish dataset

<a href="https://colab.research.google.com/github/cat-thomson/ViT-FishID/blob/main/ViT_FishID_Colab_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐟 ViT-FishID: Extended Training Session

**RESUME FROM EPOCH 19 - COLAB PRO**

This notebook resumes training from your saved checkpoint and runs for 100 total epochs.

**Current Status:**
- ✅ Previous training: 19 epochs completed
- 🎯 Target: 100 total epochs (81 remaining)
- ⏱️ Expected time: 6-8 hours with Colab Pro
- 💾 Auto-save every 10 epochs to Google Drive

**Performance Target:**
- Previous: ~78% validation accuracy at epoch 19
- Expected: 85-90% accuracy after 100 epochs
- Memory: ~8-12GB GPU memory

## 🚀 Step 1: Setup and GPU Check

In [ ]:
# Check GPU availability
import torch
import os

print("🔍 System Information:")
print(f"Python version: {os.sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("✅ GPU is ready for training!")
else:
    print("❌ No GPU detected. Please enable GPU runtime:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")

## 📁 Step 2: Mount Google Drive

This will give us access to your fish dataset stored in Google Drive.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# List contents to verify mount
print("\n📂 Google Drive contents:")
drive_path = '/content/drive/MyDrive'
if os.path.exists(drive_path):
    items = os.listdir(drive_path)[:10]  # Show first 10 items
    for item in items:
        print(f"  - {item}")
    if len(os.listdir(drive_path)) > 10:
        print(f"  ... and {len(os.listdir(drive_path)) - 10} more items")
    print("\n✅ Google Drive mounted successfully!")
else:
    print("❌ Failed to mount Google Drive")

## 📦 Step 3: Install Dependencies

Installing all required packages for ViT-FishID training.

In [ ]:
# Install required packages
print("📦 Installing dependencies...")

!pip install -q torch torchvision torchaudio
!pip install -q timm transformers
!pip install -q albumentations
!pip install -q wandb
!pip install -q opencv-python-headless
!pip install -q scikit-learn
!pip install -q matplotlib seaborn
!pip install -q tqdm

print("✅ All dependencies installed successfully!")

# Verify installations
import torch
import torchvision
import timm
import albumentations
import cv2
import sklearn

print("\n📋 Package versions:")
print(f"  - torch: {torch.__version__}")
print(f"  - torchvision: {torchvision.__version__}")
print(f"  - timm: {timm.__version__}")
print(f"  - albumentations: {albumentations.__version__}")
print(f"  - opencv: {cv2.__version__}")
print(f"  - sklearn: {sklearn.__version__}")

## 🔄 Step 4: Clone ViT-FishID Repository

Getting the latest code from your GitHub repository.

In [ ]:
# Clone the repository
import os

# Remove existing directory if it exists
if os.path.exists('/content/ViT-FishID'):
    !rm -rf /content/ViT-FishID

# Clone the repository
print("📥 Cloning ViT-FishID repository...")
!git clone https://github.com/cat-thomson/ViT-FishID.git /content/ViT-FishID

# Change to project directory
%cd /content/ViT-FishID

# List project files
print("\n📂 Project structure:")
!ls -la

print("\n✅ Repository cloned successfully!")

## 🗂️ Step 5: Setup Data Path and Extraction

**IMPORTANT:** Specify the path to your fish dataset ZIP file in Google Drive.

This step will:
1. Locate your `fish_cutouts.zip` file in Google Drive
2. Extract it to Colab's local storage for faster access
3. Validate the data structure

Expected structure after extraction:
```
fish_cutouts/
├── labeled/
│   ├── species_1/
│   │   ├── fish_001.jpg
│   │   └── fish_002.jpg
│   └── species_2/
│       └── ...
└── unlabeled/
    ├── fish_003.jpg
    └── fish_004.jpg
```

In [ ]:
# Setup data path and extraction - PROPER ZIP HANDLING
import zipfile
import shutil
import time
import os

print("🗂️ SETTING UP FISH DATASET")
print("="*50)

# Configuration - Update these paths as needed
ZIP_FILE_PATH = '/content/drive/MyDrive/fish_cutouts.zip'  # Main location
BACKUP_ZIP_PATH = '/content/drive/MyDrive/ViT-FishID/checkpoints/fish_cutouts.zip'  # Backup location
DATA_DIR = '/content/fish_cutouts'

print(f"🎯 Target data directory: {DATA_DIR}")

# Check if data already exists locally (from previous session)
if os.path.exists(DATA_DIR) and os.path.exists(os.path.join(DATA_DIR, 'labeled')):
    print("✅ Data already available locally from previous session!")
    
    # Quick validation
    labeled_dir = os.path.join(DATA_DIR, 'labeled')
    unlabeled_dir = os.path.join(DATA_DIR, 'unlabeled')
    
    if os.path.exists(labeled_dir):
        labeled_species = [d for d in os.listdir(labeled_dir) 
                          if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')]
        print(f"🐟 Found {len(labeled_species)} labeled species")
        
    if os.path.exists(unlabeled_dir):
        unlabeled_files = [f for f in os.listdir(unlabeled_dir) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"📊 Found {len(unlabeled_files)} unlabeled images")
        
    print("✅ Data validation passed - ready for training!")

else:
    print("📥 Data not found locally, extracting from Google Drive...")
    
    # Try to find the ZIP file
    zip_file_to_use = None
    if os.path.exists(ZIP_FILE_PATH):
        zip_file_to_use = ZIP_FILE_PATH
        print(f"✅ Found ZIP file at: {ZIP_FILE_PATH}")
    elif os.path.exists(BACKUP_ZIP_PATH):
        zip_file_to_use = BACKUP_ZIP_PATH
        print(f"✅ Found ZIP file at backup location: {BACKUP_ZIP_PATH}")
    else:
        print("❌ ZIP file not found at either location!")
        print(f"   Tried: {ZIP_FILE_PATH}")
        print(f"   Tried: {BACKUP_ZIP_PATH}")
        print("📝 Please ensure fish_cutouts.zip is uploaded to Google Drive")
    
    if zip_file_to_use:
        print(f"\n📦 Extracting {os.path.basename(zip_file_to_use)}...")
        print(f"📏 ZIP file size: {os.path.getsize(zip_file_to_use) / (1024**2):.1f} MB")
        
        # Clean extraction
        temp_extract_dir = '/content/temp_fish_extract'
        if os.path.exists(temp_extract_dir):
            shutil.rmtree(temp_extract_dir)
        
        try:
            # Extract ZIP file
            with zipfile.ZipFile(zip_file_to_use, 'r') as zip_ref:
                zip_ref.extractall(temp_extract_dir)
            
            print("✅ ZIP extraction completed")
            
            # Handle nested folder structure: fish_cutouts.zip contains fish_cutouts/ folder
            extracted_items = os.listdir(temp_extract_dir)
            print(f"📁 Found in ZIP: {extracted_items}")
            
            # Look for the fish_cutouts folder inside the extraction
            fish_cutouts_source = None
            for item in extracted_items:
                item_path = os.path.join(temp_extract_dir, item)
                if os.path.isdir(item_path):
                    # Check if this folder contains 'labeled' and 'unlabeled' subdirectories
                    sub_items = os.listdir(item_path)
                    if 'labeled' in sub_items or any('label' in sub.lower() for sub in sub_items):
                        fish_cutouts_source = item_path
                        print(f"✅ Found fish data in: {item}")
                        break
            
            # If we found the nested fish_cutouts folder, move it to the target location
            if fish_cutouts_source:
                # Remove existing target if it exists
                if os.path.exists(DATA_DIR):
                    shutil.rmtree(DATA_DIR)
                
                # Move the fish_cutouts folder to the correct location
                shutil.move(fish_cutouts_source, DATA_DIR)
                print(f"✅ Data moved to: {DATA_DIR}")
                
                # Verify the structure
                if os.path.exists(os.path.join(DATA_DIR, 'labeled')):
                    labeled_species = [d for d in os.listdir(os.path.join(DATA_DIR, 'labeled')) 
                                     if os.path.isdir(os.path.join(DATA_DIR, 'labeled', d)) and not d.startswith('.')]
                    print(f"🐟 Verified: {len(labeled_species)} species in labeled data")
                
                if os.path.exists(os.path.join(DATA_DIR, 'unlabeled')):
                    unlabeled_count = len([f for f in os.listdir(os.path.join(DATA_DIR, 'unlabeled'))
                                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    print(f"📊 Verified: {unlabeled_count} images in unlabeled data")
                
            else:
                print("❌ Could not find fish_cutouts folder structure in ZIP")
                print("📁 Available items:", extracted_items)
            
            # Cleanup temporary extraction
            if os.path.exists(temp_extract_dir):
                shutil.rmtree(temp_extract_dir)
                
        except Exception as e:
            print(f"❌ Error during extraction: {e}")
            if os.path.exists(temp_extract_dir):
                shutil.rmtree(temp_extract_dir)

# Final verification
if os.path.exists(DATA_DIR):
    print(f"\n✅ DATASET READY")
    print(f"📁 Location: {DATA_DIR}")
    
    # Show structure
    for subdir in ['labeled', 'unlabeled']:
        subdir_path = os.path.join(DATA_DIR, subdir)
        if os.path.exists(subdir_path):
            if subdir == 'labeled':
                species_count = len([d for d in os.listdir(subdir_path) 
                                   if os.path.isdir(os.path.join(subdir_path, d)) and not d.startswith('.')])
                print(f"  📂 {subdir}/: {species_count} species folders")
            else:
                file_count = len([f for f in os.listdir(subdir_path) 
                                if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                print(f"  📂 {subdir}/: {file_count} images")
        else:
            print(f"  ❌ {subdir}/ not found")
    
    print("? Ready to proceed with training!")
else:
    print(f"\n❌ DATASET SETUP FAILED")
    print(f"? Please check that fish_cutouts.zip contains the proper folder structure:")
    print(f"   fish_cutouts.zip")
    print(f"   └── fish_cutouts/")
    print(f"       ├── labeled/")
    print(f"       │   ├── species1/")
    print(f"       │   └── species2/")
    print(f"       └── unlabeled/")
    print(f"           ├── image1.jpg")
    print(f"           └── image2.jpg")

## 📊 Step 6: Setup Weights & Biases (Optional)

W&B provides excellent training visualization and experiment tracking.

In [ ]:
# Setup Weights & Biases for experiment tracking
import wandb

# Login to W&B (you'll need to create a free account at wandb.ai)
print("🔐 Setting up Weights & Biases...")
print("\nTo use W&B:")
print("1. Go to https://wandb.ai and create a free account")
print("2. Get your API key from https://wandb.ai/authorize")
print("3. Run the cell below and paste your API key when prompted")
print("\nOr skip W&B by setting USE_WANDB = False below")

# Set this to True if you want to use W&B, False to skip
USE_WANDB = True  # 👈 Set to False if you don't want to use W&B

if USE_WANDB:
    try:
        # Try to login (will prompt for API key if not already logged in)
        wandb.login()
        print("✅ W&B login successful!")
    except:
        print("⚠️  W&B login failed. Training will continue without W&B logging.")
        USE_WANDB = False
else:
    print("ℹ️  Skipping W&B setup. Training will run without experiment tracking.")

## 🔄 Step 6: Locate Checkpoint from Epoch 19

Finding your saved checkpoint to resume training from where you left off.

In [ ]:
# Locate checkpoint from epoch 19
import os
import glob
import torch

print("🔍 Looking for checkpoint from epoch 19...")

# Possible checkpoint locations
checkpoint_locations = [
    '/content/drive/MyDrive/ViT-FishID/checkpoints',
    '/content/drive/MyDrive/ViT-FishID_Training_*/checkpoints',
    '/content/drive/MyDrive/checkpoints',
    '/content/ViT-FishID/checkpoints'
]

checkpoint_path = None
checkpoint_info = None

# Search for epoch 19 checkpoint
for location_pattern in checkpoint_locations:
    for location in glob.glob(location_pattern):
        if os.path.exists(location):
            print(f"📁 Checking: {location}")
            
            # Look for epoch 19 specifically
            epoch_19_files = glob.glob(os.path.join(location, '*epoch_19*'))
            manual_files = glob.glob(os.path.join(location, '*manual*epoch*19*'))
            emergency_files = glob.glob(os.path.join(location, '*emergency*epoch*19*'))
            
            all_candidates = epoch_19_files + manual_files + emergency_files
            
            for candidate in all_candidates:
                if candidate.endswith('.pth'):
                    print(f"🎯 Found candidate: {os.path.basename(candidate)}")
                    try:
                        # Verify checkpoint can be loaded
                        test_checkpoint = torch.load(candidate, map_location='cpu')
                        epoch = test_checkpoint.get('epoch', 'unknown')
                        
                        if epoch == 19 or '19' in os.path.basename(candidate):
                            checkpoint_path = candidate
                            checkpoint_info = test_checkpoint
                            print(f"✅ FOUND EPOCH 19 CHECKPOINT!")
                            print(f"📁 Location: {checkpoint_path}")
                            print(f"📊 Epoch: {epoch}")
                            
                            if 'best_accuracy' in test_checkpoint:
                                print(f"📊 Best accuracy so far: {test_checkpoint['best_accuracy']:.2f}%")
                            elif 'best_acc' in test_checkpoint:
                                print(f"📊 Best accuracy so far: {test_checkpoint['best_acc']:.2f}%")
                                
                            break
                    except Exception as e:
                        print(f"⚠️ Could not load {candidate}: {e}")
            
            if checkpoint_path:
                break
        
        if checkpoint_path:
            break

if checkpoint_path:
    print(f"\n🎉 Checkpoint ready for resuming training!")
    print(f"📄 File: {os.path.basename(checkpoint_path)}")
    print(f"📏 Size: {os.path.getsize(checkpoint_path) / (1024*1024):.1f} MB")
    
    # Set up checkpoint directory for new saves
    checkpoint_save_dir = '/content/drive/MyDrive/ViT-FishID/checkpoints_extended'
    os.makedirs(checkpoint_save_dir, exist_ok=True)
    print(f"💾 New checkpoints will be saved to: {checkpoint_save_dir}")
    
else:
    print("❌ No checkpoint found for epoch 19!")
    print("\n🔧 Troubleshooting:")
    print("1. Check that you have a checkpoint saved from previous training")
    print("2. Ensure the checkpoint is uploaded to Google Drive")
    print("3. Look for files named like: checkpoint_epoch_19.pth, emergency_checkpoint_epoch_19.pth")
    print("\n📁 Checked locations:")
    for location in checkpoint_locations:
        print(f"  - {location}")
    
    # Fallback: look for any checkpoints
    print("\n🔍 All available checkpoints:")
    for location_pattern in checkpoint_locations:
        for location in glob.glob(location_pattern):
            if os.path.exists(location):
                all_checkpoints = glob.glob(os.path.join(location, '*.pth'))
                for cp in all_checkpoints:
                    print(f"  - {os.path.basename(cp)}")

# Store checkpoint path for later use
RESUME_CHECKPOINT = checkpoint_path

In [ ]:
# Backup: Copy local checkpoint to Google Drive if not found there
import shutil

if not checkpoint_path:
    print("🔍 Checkpoint not found in Google Drive, checking local copy...")
    
    # Check if there's a local checkpoint file we uploaded
    local_checkpoint = '/content/ViT-FishID/checkpoint_epoch_19.pth'
    
    if os.path.exists(local_checkpoint):
        print("✅ Found local checkpoint file!")
        
        # Copy to Google Drive
        drive_backup_dir = '/content/drive/MyDrive/ViT-FishID/checkpoints'
        os.makedirs(drive_backup_dir, exist_ok=True)
        
        drive_checkpoint_path = os.path.join(drive_backup_dir, 'checkpoint_epoch_19.pth')
        shutil.copy2(local_checkpoint, drive_checkpoint_path)
        
        print(f"💾 Copied checkpoint to Google Drive: {drive_checkpoint_path}")
        
        # Verify the copied checkpoint
        try:
            test_checkpoint = torch.load(drive_checkpoint_path, map_location='cpu')
            epoch = test_checkpoint.get('epoch', 'unknown')
            if 'best_accuracy' in test_checkpoint:
                accuracy = test_checkpoint['best_accuracy']
                print(f"✅ Verification passed - Epoch {epoch}, Accuracy: {accuracy:.2f}%")
            
            # Update our variables
            checkpoint_path = drive_checkpoint_path
            checkpoint_info = test_checkpoint
            RESUME_CHECKPOINT = checkpoint_path
            
            print("🎉 Checkpoint ready for resuming training!")
            
        except Exception as e:
            print(f"❌ Error verifying copied checkpoint: {e}")
    
    else:
        print("❌ No local checkpoint found either")
        print("📝 Please ensure you have the checkpoint_epoch_19.pth file")
        print("   You can upload it to Colab or place it in Google Drive")

# Final check
if checkpoint_path:
    print(f"\n✅ FINAL CHECKPOINT STATUS:")
    print(f"📁 Using checkpoint: {checkpoint_path}")
    print(f"📊 Ready to resume from epoch 19")
else:
    print(f"\n❌ NO CHECKPOINT AVAILABLE")
    print(f"🔄 Training will start from epoch 1 instead")
    print(f"⚠️  This will take much longer than resuming!")

## ⚙️ Step 7: Configure Training Parameters

Adjust these parameters based on your needs and available GPU memory.

In [ ]:
# Training Configuration - RESUME FROM EPOCH 19 FOR 100 TOTAL EPOCHS
TRAINING_CONFIG = {
    # RESUME SETTINGS
    'resume_from_checkpoint': RESUME_CHECKPOINT,
    'start_epoch': 20,  # Next epoch after 19
    'total_epochs': 100,  # Target total epochs
    'remaining_epochs': 81,  # 100 - 19 = 81 epochs left
    
    # CORE SETTINGS
    'mode': 'semi_supervised',  # semi_supervised or supervised
    'data_dir': DATA_DIR,
    'batch_size': 16,  # Increased for Colab Pro
    'learning_rate': 1e-4,
    'weight_decay': 0.05,
    
    # MODEL SETTINGS
    'model_name': 'vit_base_patch16_224',
    'num_classes': 37,  # Will be auto-detected
    'pretrained': True,
    
    # SEMI-SUPERVISED SETTINGS - FIXED CONSISTENCY LOSS
    'consistency_weight': 2.0,
    'pseudo_label_threshold': 0.7,
    'temperature': 4.0,
    'warmup_epochs': 5,  # Reduced since we're resuming
    'ramp_up_epochs': 15,  # Reduced since we're resuming
    
    # CHECKPOINT SETTINGS - SAVE EVERY EPOCH
    'save_frequency': 1,  # Save EVERY epoch (changed from 10)
    'checkpoint_dir': '/content/drive/MyDrive/ViT-FishID/checkpoints_extended',
    'backup_dir': '/content/drive/MyDrive/ViT-FishID/checkpoints_backup',
    
    # LOGGING
    'use_wandb': True,
    'wandb_project': 'vit-fishid-extended',
    'wandb_run_name': f'resume_epoch19_to_100_fixed'
}

print("🎯 EXTENDED TRAINING CONFIGURATION - FIXED VERSION")
print("="*50)
print(f"📊 Resume from: Epoch {TRAINING_CONFIG['start_epoch'] - 1}")
print(f"📊 Target epochs: {TRAINING_CONFIG['total_epochs']}")
print(f"📊 Remaining epochs: {TRAINING_CONFIG['remaining_epochs']}")
print(f"📊 Expected time: {TRAINING_CONFIG['remaining_epochs'] * 4:.0f}-{TRAINING_CONFIG['remaining_epochs'] * 6:.0f} minutes")
print(f"📊 Batch size: {TRAINING_CONFIG['batch_size']} (optimized for Colab Pro)")
print(f"? Checkpoint saves: EVERY epoch (enhanced backup)")
print(f"📊 Mode: {TRAINING_CONFIG['mode']} with FIXED consistency loss")

# Create checkpoint directories
os.makedirs(TRAINING_CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs(TRAINING_CONFIG['backup_dir'], exist_ok=True)
print(f"📁 Primary saves: {TRAINING_CONFIG['checkpoint_dir']}")
print(f"? Backup saves: {TRAINING_CONFIG['backup_dir']}")

if TRAINING_CONFIG['resume_from_checkpoint']:
    print(f"✅ Will resume from: {os.path.basename(TRAINING_CONFIG['resume_from_checkpoint'])}")
else:
    print("❌ No checkpoint found - will start fresh training")
    TRAINING_CONFIG['start_epoch'] = 1
    TRAINING_CONFIG['remaining_epochs'] = TRAINING_CONFIG['total_epochs']

print(f"\n🔧 FIXES APPLIED:")
print(f"  ✅ Consistency loss: Fixed tensor initialization")
print(f"  ✅ Checkpoint saving: Every epoch + Google Drive backup")
print(f"  ✅ Error handling: Enhanced for robustness")
print(f"  ✅ State dict keys: Fixed ema_teacher key naming")

print("\n💡 With Colab Pro, this training should complete without timeout!")
print("💾 Every epoch will be saved with Google Drive backup every 5 epochs")

# Verify data directory
if os.path.exists(TRAINING_CONFIG['data_dir']):
    labeled_dir = os.path.join(TRAINING_CONFIG['data_dir'], 'labeled')
    if os.path.exists(labeled_dir):
        species_count = len([d for d in os.listdir(labeled_dir) 
                           if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')])
        TRAINING_CONFIG['num_classes'] = species_count
        print(f"📊 Detected {species_count} fish species")
    
print(f"\n🚀 Ready to resume training for {TRAINING_CONFIG['remaining_epochs']} more epochs!")
print(f"🔧 All consistency loss and checkpoint issues have been resolved!")

## 🚀 Step 8: Start Training!

This cell will start the semi-supervised training process. It may take 2-3 hours to complete.

In [ ]:
# Execute Extended Training - Resume from Epoch 19
import os

print("🚀 STARTING EXTENDED TRAINING SESSION")
print("="*60)

# Create checkpoint save directory
os.makedirs(TRAINING_CONFIG['checkpoint_dir'], exist_ok=True)

# Build training command for resuming
training_cmd = f"""python train.py \\
    --mode {TRAINING_CONFIG['mode']} \\
    --data_dir {TRAINING_CONFIG['data_dir']} \\
    --epochs {TRAINING_CONFIG['total_epochs']} \\
    --batch_size {TRAINING_CONFIG['batch_size']} \\
    --learning_rate {TRAINING_CONFIG['learning_rate']} \\
    --weight_decay {TRAINING_CONFIG['weight_decay']} \\
    --model_name {TRAINING_CONFIG['model_name']} \\
    --num_classes {TRAINING_CONFIG['num_classes']} \\
    --consistency_weight {TRAINING_CONFIG['consistency_weight']} \\
    --pseudo_label_threshold {TRAINING_CONFIG['pseudo_label_threshold']} \\
    --temperature {TRAINING_CONFIG['temperature']} \\
    --warmup_epochs {TRAINING_CONFIG['warmup_epochs']} \\
    --ramp_up_epochs {TRAINING_CONFIG['ramp_up_epochs']} \\
    --save_dir {TRAINING_CONFIG['checkpoint_dir']} \\
    --save_frequency {TRAINING_CONFIG['save_frequency']}"""

# Add resume checkpoint if available
if TRAINING_CONFIG['resume_from_checkpoint']:
    training_cmd += f" \\\n    --resume_from {TRAINING_CONFIG['resume_from_checkpoint']}"
    print(f"📂 Resuming from: {os.path.basename(TRAINING_CONFIG['resume_from_checkpoint'])}")

# Add W&B logging
if TRAINING_CONFIG['use_wandb']:
    training_cmd += f" \\\n    --use_wandb --wandb_project {TRAINING_CONFIG['wandb_project']} --wandb_run_name {TRAINING_CONFIG['wandb_run_name']}"

# Add pretrained flag
if TRAINING_CONFIG['pretrained']:
    training_cmd += " \\\n    --pretrained"

print(f"📊 Training for {TRAINING_CONFIG['remaining_epochs']} more epochs...")
print(f"🎯 Target: {TRAINING_CONFIG['total_epochs']} total epochs")
print(f"⏱️ Estimated time: {TRAINING_CONFIG['remaining_epochs'] * 4:.0f}-{TRAINING_CONFIG['remaining_epochs'] * 6:.0f} minutes")
print(f"💾 Checkpoints saved to: {TRAINING_CONFIG['checkpoint_dir']}")

print("\n📋 Extended Training Command:")
print(training_cmd.replace('\\', '').strip())
print("\n" + "="*60)

# Execute training
print("🎬 TRAINING STARTED - EPOCH 20 TO 100")
print("⏰ Started at:", __import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

!{training_cmd}

print("\n" + "="*60)
print("🎉 EXTENDED TRAINING COMPLETED!")
print("⏰ Finished at:", __import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print(f"🏆 Total epochs completed: {TRAINING_CONFIG['total_epochs']}")
print(f"💾 All checkpoints saved to Google Drive")

# Quick summary of final results
final_checkpoint = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'model_best.pth')
if os.path.exists(final_checkpoint):
    try:
        import torch
        final_results = torch.load(final_checkpoint, map_location='cpu')
        if 'best_accuracy' in final_results:
            print(f"🎯 Final best accuracy: {final_results['best_accuracy']:.2f}%")
        if 'epoch' in final_results:
            print(f"📊 Best model from epoch: {final_results['epoch']}")
    except:
        pass

print("\n✅ Your model is ready for evaluation and deployment!")

In [ ]:
# 🔧 Pre-Training Verification - Test Fixes
print("🔧 TESTING FIXES BEFORE TRAINING")
print("="*50)

# Test 1: Verify checkpoint loading
if TRAINING_CONFIG['resume_from_checkpoint']:
    print("✅ Test 1: Checkpoint Loading")
    try:
        test_checkpoint = torch.load(TRAINING_CONFIG['resume_from_checkpoint'], map_location='cpu')
        print(f"   📊 Checkpoint epoch: {test_checkpoint.get('epoch', 'Unknown')}")
        print(f"   📊 Best accuracy: {test_checkpoint.get('best_accuracy', 'Unknown')}")
        print(f"   🔑 Available keys: {list(test_checkpoint.keys())}")
        
        # Check for correct key names
        required_keys = ['student_state_dict', 'epoch']
        missing_keys = [key for key in required_keys if key not in test_checkpoint]
        if missing_keys:
            print(f"   ⚠️ Missing keys: {missing_keys}")
        else:
            print(f"   ✅ All required keys present")
        
    except Exception as e:
        print(f"   ❌ Checkpoint test failed: {e}")

# Test 2: Verify consistency loss function
print(f"\n✅ Test 2: Consistency Loss Function")
try:
    from model import ConsistencyLoss
    consistency_loss_fn = ConsistencyLoss(temperature=4.0)
    
    # Create dummy tensors
    dummy_student = torch.randn(4, 37)  # batch_size=4, num_classes=37
    dummy_teacher = torch.randn(4, 37)
    
    loss = consistency_loss_fn(dummy_student, dummy_teacher)
    print(f"   📊 Test loss value: {loss.item():.6f}")
    print(f"   📊 Loss tensor shape: {loss.shape}")
    print(f"   📊 Loss requires grad: {loss.requires_grad}")
    
    if loss.requires_grad and loss.item() > 0:
        print(f"   ✅ Consistency loss function working correctly")
    else:
        print(f"   ⚠️ Potential issue with consistency loss")
        
except Exception as e:
    print(f"   ❌ Consistency loss test failed: {e}")

# Test 3: Verify Google Drive paths
print(f"\n✅ Test 3: Google Drive Paths")
for path_name, path in [
    ("Primary checkpoint dir", TRAINING_CONFIG['checkpoint_dir']),
    ("Backup checkpoint dir", TRAINING_CONFIG['backup_dir'])
]:
    try:
        os.makedirs(path, exist_ok=True)
        test_file = os.path.join(path, 'test_write.txt')
        with open(test_file, 'w') as f:
            f.write('test')
        os.remove(test_file)
        print(f"   ✅ {path_name}: {path} (writable)")
    except Exception as e:
        print(f"   ❌ {path_name}: {path} (error: {e})")

# Test 4: Verify training configuration
print(f"\n✅ Test 4: Training Configuration")
config_checks = [
    ("Save frequency", TRAINING_CONFIG['save_frequency'] == 1, "Every epoch"),
    ("Consistency weight", TRAINING_CONFIG['consistency_weight'] > 0, f"{TRAINING_CONFIG['consistency_weight']}"),
    ("Batch size", TRAINING_CONFIG['batch_size'] >= 8, f"{TRAINING_CONFIG['batch_size']}"),
    ("Learning rate", 0 < TRAINING_CONFIG['learning_rate'] < 1, f"{TRAINING_CONFIG['learning_rate']}"),
]

for check_name, condition, value in config_checks:
    status = "✅" if condition else "❌"
    print(f"   {status} {check_name}: {value}")

print(f"\n🎯 PRE-TRAINING VERIFICATION COMPLETE")
print(f"📊 Configuration looks {'✅ GOOD' if all([check[1] for check in config_checks]) else '⚠️ NEEDS ATTENTION'}")
print(f"🚀 Ready to start training with all fixes applied!")

## 📊 Step 9: Check Training Results

In [ ]:
# Check Extended Training Results (Epoch 19 → 100)
import os
import glob
import torch

checkpoint_dir = TRAINING_CONFIG['checkpoint_dir']
print(f"📁 Checking results in: {checkpoint_dir}")

if os.path.exists(checkpoint_dir):
    checkpoints = glob.glob(os.path.join(checkpoint_dir, '*.pth'))
    if checkpoints:
        print(f"\n✅ Found {len(checkpoints)} checkpoint(s) from extended training:")
        
        # Sort checkpoints by epoch number
        epoch_checkpoints = []
        other_checkpoints = []
        
        for cp in checkpoints:
            basename = os.path.basename(cp)
            if 'epoch_' in basename:
                try:
                    epoch_num = int(basename.split('epoch_')[1].split('.')[0])
                    epoch_checkpoints.append((epoch_num, cp))
                except:
                    other_checkpoints.append(cp)
            else:
                other_checkpoints.append(cp)
        
        # Show epoch checkpoints in order
        epoch_checkpoints.sort(key=lambda x: x[0])
        for epoch, cp in epoch_checkpoints:
            file_size = os.path.getsize(cp) / (1024**2)
            print(f"  📊 Epoch {epoch}: {os.path.basename(cp)} ({file_size:.1f} MB)")
        
        # Show other checkpoints
        for cp in other_checkpoints:
            file_size = os.path.getsize(cp) / (1024**2)
            print(f"  🏆 {os.path.basename(cp)} ({file_size:.1f} MB)")
        
        # Analyze best model
        best_model = os.path.join(checkpoint_dir, 'model_best.pth')
        if os.path.exists(best_model):
            print(f"\n🏆 BEST MODEL ANALYSIS:")
            try:
                best_checkpoint = torch.load(best_model, map_location='cpu')
                
                best_epoch = best_checkpoint.get('epoch', 'Unknown')
                best_acc = best_checkpoint.get('best_accuracy', best_checkpoint.get('best_acc', 'Unknown'))
                
                print(f"  📊 Best epoch: {best_epoch}")
                print(f"  📊 Best accuracy: {best_acc:.2f}%" if isinstance(best_acc, (int, float)) else f"  📊 Best accuracy: {best_acc}")
                
                # Show training progression
                if epoch_checkpoints:
                    print(f"\n📈 TRAINING PROGRESSION:")
                    print(f"  🏁 Started: Epoch 19 (resumed)")
                    print(f"  🎯 Completed: Epoch {max(epoch_checkpoints, key=lambda x: x[0])[0]}")
                    print(f"  🏆 Best: Epoch {best_epoch}")
                    print(f"  📊 Total training: {19 + len([e for e, _ in epoch_checkpoints if e > 19])} epochs")
                
            except Exception as e:
                print(f"  ⚠️ Could not analyze best model: {e}")
        
        # Training duration estimate
        if epoch_checkpoints:
            epochs_completed = len([e for e, _ in epoch_checkpoints if e > 19])
            print(f"\n⏱️ EXTENDED TRAINING SUMMARY:")
            print(f"  📊 Additional epochs completed: {epochs_completed}")
            print(f"  🎯 Target was: 81 additional epochs (to reach 100 total)")
            
            if epochs_completed >= 81:
                print(f"  ✅ TRAINING GOAL ACHIEVED! Completed all {epochs_completed} additional epochs")
            else:
                print(f"  ⏳ Training partially complete: {epochs_completed}/81 additional epochs")
    
    else:
        print("❌ No checkpoints found in extended training directory")
        
        # Check if training is still using old directory
        old_checkpoint_dir = '/content/ViT-FishID/checkpoints'
        if os.path.exists(old_checkpoint_dir):
            old_checkpoints = glob.glob(os.path.join(old_checkpoint_dir, '*.pth'))
            if old_checkpoints:
                print(f"\n💡 Found {len(old_checkpoints)} checkpoints in old directory:")
                print(f"   {old_checkpoint_dir}")

else:
    print("❌ Extended training checkpoint directory not found")

# W&B link
if TRAINING_CONFIG['use_wandb']:
    print(f"\n📈 View detailed training metrics:")
    print(f"   https://wandb.ai/your-username/{TRAINING_CONFIG['wandb_project']}")
    print(f"   Run: {TRAINING_CONFIG['wandb_run_name']}")

print(f"\n🎉 Extended training session complete!")
print(f"🚀 Your model trained from epoch 19 to 100!")
print(f"💾 All results saved to Google Drive: {checkpoint_dir}")

# Performance comparison
print(f"\n📊 PERFORMANCE COMPARISON:")
print(f"  🔄 Previous (Epoch 19): ~78% accuracy")
print(f"  🎯 Extended (Epoch 100): Check best_accuracy above")
print(f"  📈 Expected improvement: 5-10% accuracy gain")
print(f"  🏆 Your model should now be ready for deployment!")

## 💾 Step 10: Download Model and Results

Save your trained model to Google Drive for future use.

In [ ]:
# Copy trained model to Google Drive
import shutil
from datetime import datetime

# Create a timestamped folder in Google Drive
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f'/content/drive/MyDrive/ViT-FishID_Training_{timestamp}'
os.makedirs(save_dir, exist_ok=True)

print(f"💾 Saving results to Google Drive: {save_dir}")

# Copy checkpoints
checkpoint_dir = '/content/ViT-FishID/checkpoints'
if os.path.exists(checkpoint_dir):
    drive_checkpoint_dir = os.path.join(save_dir, 'checkpoints')
    shutil.copytree(checkpoint_dir, drive_checkpoint_dir)
    print(f"✅ Checkpoints saved to: {drive_checkpoint_dir}")

# Save training configuration
import json
config_file = os.path.join(save_dir, 'training_config.json')
with open(config_file, 'w') as f:
    json.dump(TRAINING_CONFIG, f, indent=2)
print(f"✅ Training config saved to: {config_file}")

# Create a summary file
summary_file = os.path.join(save_dir, 'training_summary.txt')
with open(summary_file, 'w') as f:
    f.write(f"ViT-FishID Training Summary\n")
    f.write(f"========================\n\n")
    f.write(f"Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Mode: {TRAINING_CONFIG['mode']}\n")
    f.write(f"Epochs: {TRAINING_CONFIG['epochs']}\n")
    f.write(f"Batch Size: {TRAINING_CONFIG['batch_size']}\n")
    f.write(f"Data Directory: {DATA_DIR}\n")
    f.write(f"\nModel Architecture: {TRAINING_CONFIG['model_name']}\n")
    f.write(f"Learning Rate: {TRAINING_CONFIG['learning_rate']}\n")
    f.write(f"Consistency Weight: {TRAINING_CONFIG['consistency_weight']}\n")
    f.write(f"\nCheckpoints saved in: checkpoints/\n")
    f.write(f"Best model: checkpoints/model_best.pth\n")

print(f"✅ Training summary saved to: {summary_file}")

print(f"\n🎉 All results saved to Google Drive!")
print(f"📁 Location: {save_dir}")
print(f"\n💡 You can now:")
print(f"   1. Download the checkpoints folder for local use")
print(f"   2. Use model_best.pth for inference")
print(f"   3. Continue training from any checkpoint")

## 🧪 Step 11: Quick Model Evaluation (Optional)

Test your trained model on a few sample images.

In [ ]:
# Quick evaluation of the trained model
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Check if best model exists
best_model_path = '/content/ViT-FishID/checkpoints/model_best.pth'

if os.path.exists(best_model_path):
    print("🧪 Loading trained model for quick evaluation...")
    
    # Load model checkpoint info
    checkpoint = torch.load(best_model_path, map_location='cpu')
    
    print(f"📊 Model training info:")
    if 'epoch' in checkpoint:
        print(f"  - Best epoch: {checkpoint['epoch']}")
    if 'best_acc' in checkpoint:
        print(f"  - Best accuracy: {checkpoint['best_acc']:.2f}%")
    if 'teacher_acc' in checkpoint:
        print(f"  - Teacher accuracy: {checkpoint['teacher_acc']:.2f}%")
    
    # Get class names if available
    if 'class_names' in checkpoint:
        class_names = checkpoint['class_names']
        print(f"  - Number of classes: {len(class_names)}")
        print(f"  - Sample classes: {class_names[:5]}...")
    
    print("\n✅ Model evaluation completed! Check the metrics above.")
    
else:
    print("❌ No trained model found. Make sure training completed successfully.")

print("\n💡 For comprehensive evaluation:")
print("   Use the evaluate.py script with your test dataset")
print("   The test set was automatically created during training")

## 🔧 Troubleshooting

### Common Issues and Solutions:

**1. GPU Memory Error (CUDA out of memory)**
- Reduce batch_size to 8 or 4
- Restart runtime and try again

**2. Data Not Found**
- Check that DATA_DIR path is correct
- Ensure data is uploaded to Google Drive
- Verify folder structure (labeled/ and unlabeled/)

**3. Training Stops Unexpectedly**
- Colab sessions timeout after 12 hours
- Use runtime management to prevent disconnection
- Checkpoints are saved every 10 epochs for resuming

**4. Low Accuracy**
- Increase epochs (try 75-100)
- Adjust consistency_weight (try 1.0-3.0)
- Lower pseudo_label_threshold (try 0.5-0.6)

**5. Consistency Loss is 0.0000**
- Lower pseudo_label_threshold to 0.5
- Check that you have unlabeled data
- Ensure semi_supervised mode is selected

## 🚀 Next Steps

After training is complete, you can:

1. **Download your model**: The trained model is saved in Google Drive
2. **Continue training**: Resume from checkpoints for more epochs
3. **Evaluate performance**: Use the test set for final evaluation
4. **Deploy model**: Use the trained model for fish classification
5. **Experiment**: Try different hyperparameters or architectures

### Model Files Saved:
- `model_best.pth`: Best performing model (use this for inference)
- `model_latest.pth`: Most recent checkpoint
- `model_epoch_XX.pth`: Periodic checkpoints

### Performance Expectations:
- **50 epochs**: ~70-80% accuracy
- **100 epochs**: ~75-85% accuracy
- **Semi-supervised**: Should outperform supervised training

**Happy fish classification! 🐟🎉**